In [1]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

# Data

In [4]:
import numpy as np
import os, sys
import random #shuffle
from netCDF4 import Dataset

import numpy as np
from netCDF4 import Dataset

def CMIPdata(Xdata, Ydata, out , myform):
    inp1 = Dataset(Xdata,'r')
    inp2 = Dataset(Ydata,'r')
    sst_1 = np.zeros((myform,140,12,24,72))

    #首年序列
    #i= 21个模式
    for i in range(myform):
        sst_1[i,:,:,:,:] = inp1.variables['sst1'][1+141*i:141+141*i,0:12,:,:]
    #(21,140,12,24,72)
    #首年序列

    
    #flatted扁平化                                             
    sst_2 = np.zeros((myform,1680,24,72))
    for i in range(myform):
        for j in range(140):
            sst_2[i,j*12:(j+1)*12,:,:] = sst_1[i,j,:,:,:]
    
        #(21,1680,24,72)
        #每个模式 139*12+1 = 1680 个序列 序列长度为12

    #丢弃1个月 1862.1
    sst_2 = sst_2[:,1:,:,:]
    #(form,1679,24,72)


    winsize = 12
    #winnum = 1679-winsize +1 #1668
    """跳跃六个月滑窗 = 6"""
    ts=6 
    #(1679-12)/6+1 =27
    winnum=int((1679-12)/ts +1) 
    
    sst_3 = np.zeros((myform,winnum,12,24,72))
    for i in range(myform):
        for j in range(winnum):
            #print(j)
            sst_3[i,j,:,:,:] = sst_2[i,(j*ts):(j*ts)+winsize,:,:]
            

    sst_4 = np.zeros((int(winnum*myform),12,24,72))
    #t300_4 = np.zeros((int(winnum*myform),12,24,72))
    for i in range(myform):
        sst_4[i*winnum:(i+1)*winnum,:,:,:] = sst_3[i,:,:,:,:]


    #sst_4 = sst_4.swapaxes(1, 3)
    trX = sst_4
    del sst_1,sst_2,sst_3,sst_4
    #trX = trX.reshape(int(winnum*myform),12,1728)

    #保存np数组
    #(5859, 12, 24, 72, 2)
    #np.save("./CMIPdata/CMIP_trX_21_ts6_out.npy",trX) 

    #label
    #平铺
    pr_1 = np.zeros(((myform),1692))#21,1692
    for i in range(myform):
        for j in range(141):
            pr_1[i,j*12:(j+1)*12] = inp2.variables['pr'][(141*i)+j,:,0,0]
            
    #out
    pr_2 =  np.zeros((myform,winnum,out))#form,1668,1
    for i in range(myform):
        for j in range(winnum):
            pr_2[i,j,:] = pr_1[i,(j*ts):(j*ts)+out]

    trY = np.zeros((int(winnum*myform),out))#form*1668,1
    for i in range(myform):
        trY[i*winnum:(i+1)*winnum,:] = pr_2[i,:,:]

    #np.save("./CMIPdata/CMIP_trY_21_ts6_out%s.npy"%out,trY)
    trY_decoder_input = np.zeros((int(winnum*myform),out))
    trY_decoder_input[:,1:] = trY[:,:-1]

    #return trX[1:,:,:] ,trY.reshape(-1,24,1)[1:,:,:] ,trY_decoder_input
    return trX ,trY.reshape(-1,24,1) ,trY_decoder_input.reshape(-1,24,1)

def GOSDAdata(Xdata, Ydata, out):
    #test data
    inp11 = Dataset(Xdata,'r')
    inp22 = Dataset(Ydata,'r')
                                              
    sst_11 = np.zeros((33,12,24,72)) #1983-2015
    #t300_11 = np.zeros((33,12,24,72))

    sst_11[:,:,:,:] = inp11.variables['sst'][3:,0:12,:,:]
    #t300_11[:,:,:,:] = inp11.variables['t300'][3:,0:12,:,:]
    #(33,12,24,72)

    sst_22 = np.zeros((396,24,72))
    #t300_22 = np.zeros((396,24,72))

    for i in range(33):
        sst_22[i*12:(i+1)*12,:,:] = sst_11[i,:,:,:]
        #t300_22[i*12:(i+1)*12,:,:] = t300_11[i,:,:,:]
    #(396,24,72)
    #丢弃一个月，便于制作滑窗序列
    sst_22 = sst_22[1:,:,:]
    #t300_22 = t300_22[1:,:,:]#(395,24,72)

    #滑窗
    winsize = 12
    winnum = 395-winsize + 1 #384
    sst_33 = np.zeros((winnum, winsize, 24, 72))#(384, 12, 24, 72)
    #t300_33 = np.zeros((winnum, winsize, 24, 72))

    for i in range(winnum):
        sst_33[i,:,:,:] = sst_22[i:i+12,:,:]
        #t300_33[i,:,:,:] = t300_22[i:i+12,:,:]
    #(384,12,24,72)


    #channel = 2
    #testX = np.zeros((winnum,12,24,72,2))#384
    #testX[:,:,:,:,0] = sst_33
    #testX[:,:,:,:,1] = t300_33
    #sst_33 = sst_33.swapaxes(1, 3)

    testX = sst_33#384
    #testX[:,:,:,1] = t300_33.reshape(384,12,1728)
    del sst_11,sst_22,sst_33
    #del t300_11,t300_22,t300_33
    #testX = testX.reshape(384,12,3456)

    #label
    inpv22 = np.zeros((408))#2017-1984 +1=34
    for i in range(34):
        inpv22[i*12:(i+1)*12] = inp22.variables['pr'][i+2,:,0,0]# +2:从1984开始
        #(408)

    testY = np.zeros((winnum,out,1))
    #out = 1
    #滑窗
    for i in range(winnum):
        testY[i,:,0] = inpv22[i:i+out]#24
    #(384,out,1)

    return testX, testY

def SODAdata(Xdata, Ydata, out):
    inp1 = Dataset(Xdata,'r')
    inp2 = Dataset(Ydata,'r')

    #time_step = 1 month                                               
    sst_1 = np.zeros((99,12,24,72))
    #t300_1 = np.zeros((99,12,24,72))

    sst_1[:,:,:,:] = inp1.variables['sst'][1:,0:12,:,:]
    #sst_1[:,:,:,:] = inp1.variables['t300'][1:,0:12,:,:]
    #(99,12,24,72)
    #1872-1970

    sst_2 = np.zeros((1188,24,72))
    #t300_2 = np.zeros((1188,24,72))

    for i in range(99):
        sst_2[i*12:(i+1)*12,:,:] = sst_1[i,:,:,:]
        #t300_2[i*12:(i+1)*12,:,:] = t300_1[i,:,:,:]
    #(1188,24,72)
    #丢弃一个月，便于制作滑窗序列
    sst_2 = sst_2[1:,:,:]
    #t300_2 = t300_2[1:,:,:]#(1187,24,72)

    sst_3 = np.zeros((1176,12,24,72))
    #t300_3 = np.zeros((1176,12,24,72))
    #滑窗
    for i in range(1176):
        sst_3[i:,:,:] = sst_2[i:i+12,:,:]
        #t300_3[i,:,:,:] = sst_2[i:i+12,:,:]
    #(1176,12,24,72)
    trX = sst_3

    #channel = 2
    #trX = np.zeros((1176,12,24,72,2))
    #trX[:,:,:,:,0] = sst_3
    #trX[:,:,:,:,1] = t300_3

    #保存np数组
    #np.save("./SODAdata/SODA_trX_ts1_out.npy",trX)

    #label
    inpv2 = np.zeros((1200))
    for i in range(100):
        inpv2[i*12:(i+1)*12] = inp2.variables['pr'][i,:,0,0]
    #(1200)

    #out = 1
    trY = np.zeros((1176,out,1))
    #滑窗
    for i in range(1176):
        trY[i,:,0] = inpv2[i:i+out]#24
    #(1176,24)
    
    
    trY_decoder_input = np.zeros((1176,out,1))
    trY_decoder_input[:,1:,:] = trY[:,:-1,:]

    #np.save("./SODAdata/SODA_trY_ts1_out%s.npy"%out,trY) 
    return trX , trY, trY_decoder_input

trX, trY, trY_decoder_input = CMIPdata('/home/d/Q/liyanqiu/saconvlstm/CMIP5.input.36mon.1861_2001.nc',\
                                          '/home/d/Q/liyanqiu/saconvlstm/CMIP5.label.12mon.1863_2003.nc', 24,  21)

testX, testY = GOSDAdata('/home/d/Q/liyanqiu/saconvlstm/GODAS.input.36mon.1980_2015.nc',\
                         '/home/d/Q/liyanqiu/saconvlstm/GODAS.label.12mon.1982_2017.nc', 24)

trX_SODA, trY_SODA ,trY_decoder_input_SODA = SODAdata('/home/d/Q/liyanqiu/saconvlstm/SODA.input.36mon.1871_1970.nc',
                              '/home/d/Q/liyanqiu/saconvlstm/SODA.label.12mon.1873_1972.nc',24)

indices = np.arange(trX.shape[0])
np.random.shuffle(indices)
trX = trX[indices]
trY = trY[indices]
trY_decoder_input = trY_decoder_input[indices]

# shuffle
indices_soda = np.arange(trX_SODA.shape[0])
np.random.shuffle(indices_soda)
trX_SODA = trX_SODA[indices_soda]
trY_SODA = trY_SODA[indices_soda]
trY_decoder_input_SODA = trY_decoder_input_SODA[indices_soda]

class MyDataSet(Data.Dataset):
    """自定义DataLoader"""

    def __init__(self, enc_inputs, dec_inputs, dec_outputs):
        super(MyDataSet, self).__init__()
        self.enc_inputs = enc_inputs
        self.dec_inputs = dec_inputs
        self.dec_outputs = dec_outputs

    def __len__(self):
        return self.enc_inputs.shape[0]

    def __getitem__(self, idx):
        return self.enc_inputs[idx], self.dec_inputs[idx], self.dec_outputs[idx]
    
batch_size = 128
trainsize = 2500 #选择cmip训练数量
trX = trX[:trainsize]
trY = trY[:trainsize]
trY_decoder_input = trY_decoder_input[:trainsize]

#trY = trY[:, 5:8, :]
#trY_decoder_input = trY_decoder_input[:, 5:8, :]

split_index = int(trX.shape[0]*0.8)
split_index_soda = int(trX_SODA.shape[0]*0.8)
print("split_index_soda:" + str(split_index_soda))

print("split_index:" + str(split_index))
loader_train = Data.DataLoader(
    MyDataSet(
        trX[:split_index].astype(np.float32), 
        trY_decoder_input[:split_index].astype(np.float32), 
        trY[:split_index].astype(np.float32)), 
    batch_size, shuffle = True)
loader_valid = Data.DataLoader(
    MyDataSet(
        trX[split_index:].astype(np.float32), 
        trY_decoder_input[split_index:].astype(np.float32), 
        trY[split_index:].astype(np.float32)), 
    batch_size, shuffle=False)

loader_train_sado = Data.DataLoader(
    MyDataSet(
        trX_SODA[:split_index_soda].astype(np.float32), 
        trY_decoder_input_SODA[:split_index_soda].astype(np.float32), 
        trY_SODA[:split_index_soda].astype(np.float32)), 
    batch_size, shuffle = True)
loader_valid_soda = Data.DataLoader(
    MyDataSet(
        trX_SODA[split_index_soda:].astype(np.float32), 
        trY_decoder_input_SODA[split_index_soda:].astype(np.float32), 
        trY_SODA[split_index_soda:].astype(np.float32)), 
    batch_size, shuffle=False)

print(trX.shape)
print(trY_decoder_input.shape)
print(trY.shape)
print(testX.shape)
print(len(testX))
print(trX_SODA.shape)
print(trY_decoder_input_SODA.shape)
print(trY_SODA.shape)

split_index_soda:940
split_index:2000
(2500, 12, 24, 72)
(2500, 24, 1)
(2500, 24, 1)
(384, 12, 24, 72)
384
(1176, 12, 24, 72)
(1176, 24, 1)
(1176, 24, 1)


# Model

In [3]:
device = 'cuda'

src_len = 12
tgt_len = 24
tgt_vocab_size = 1 # out unit

filters = 30 #50
d_model = 540 #900  # Embedding Size（token embedding和position编码的维度） 加入CNN！！！
#d_model = 24*72
d_ff = 2048  # FeedForward dimension (两次线性层中的隐藏层 512->2048->512，线性层是用来做特征提取的），当然最后会再接一个projection层
d_k = d_v = 64  # dimension of K(=Q), V（Q和K的维度需要相同，这里为了方便让K=V）
encoder_n_layers = 1  # number of Encoder  Layer（Block的个数）
decoder_n_layers = 1  # # number of  Decoder Layer（Block的个数）
n_heads = 1 # number of heads in Multi-Head Attention（有几套头）

def get_attn_pad_mask(seq_q, seq_k):
    # pad mask的作用：在对value向量加权平均的时候，可以让pad对应的alpha_ij=0，这样注意力就不会考虑到pad向量
    """这里的q,k表示的是两个序列（跟注意力机制的q,k没有关系），例如encoder_inputs (x1,x2,..xm)和encoder_inputs (x1,x2..xm)
    encoder和decoder都可能调用这个函数，所以seq_len视情况而定
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    """
    batch_size, len_q, = seq_q.size(0), seq_q.size(1)  # 这个seq_q只是用来expand维度的
    batch_size, len_k, = seq_k.size(0), seq_k.size(1)
    # eq(zero) is PAD token
    a = torch.ones(batch_size, 1 ,len_k)
    pad_attn_mask = a.data.eq(0)
    return pad_attn_mask.expand(batch_size, len_q, len_k).to(device)  # [batch_size, len_q, len_k] 构成一个立方体(batch_size个这样的矩阵)

def get_attn_subsequence_mask(seq):
    """
    seq: [batch_size, tgt_len]
    """
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    # attn_shape: [batch_size, tgt_len, tgt_len]
    subsequence_mask = np.triu(np.ones(attn_shape), k=1)  # 生成一个上三角矩阵
    subsequence_mask = torch.from_numpy(subsequence_mask).byte()
    # print(subsequence_mask.data)
    return subsequence_mask.to(device)  # [batch_size, tgt_len, tgt_len]

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        x: [seq_len, batch_size, d_model]
        """
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        """
        Q: [batch_size, n_heads, len_q, d_k]
        K: [batch_size, n_heads, len_k, d_k]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
        说明：在encoder-decoder的Attention层中len_q(q1,..qt)和len_k(k1,...km)可能不同
        """
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)  # scores : [batch_size, n_heads, len_q, len_k]
        # mask矩阵填充scores（用-1e9填充scores中与attn_mask中值为1位置相对应的元素）
        scores.masked_fill_(attn_mask, -1e9)  # Fills elements of self tensor with value where mask is True.

        attn = nn.Softmax(dim=-1)(scores)  # 对最后一个维度(v)做softmax
        # scores : [batch_size, n_heads, len_q, len_k] * V: [batch_size, n_heads, len_v(=len_k), d_v]
        context = torch.matmul(attn, V)  # context: [batch_size, n_heads, len_q, d_v]
        # context：[[z1,z2,...],[...]]向量, attn注意力稀疏矩阵（用于可视化的）
        return context, attn
    
class MultiHeadAttention(nn.Module):
    """这个Attention类可以实现:
    Encoder的Self-Attention
    Decoder的Masked Self-Attention
    Encoder-Decoder的Attention
    """

    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)  # q,k必须维度相同，不然无法做点积
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)

    def forward(self, input_Q, input_K, input_V, attn_mask):
        """
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        """
        residual, batch_size = input_Q, input_Q.size(0)
        # 下面的多头的参数矩阵是放在一起做线性变换的，然后再拆成多个头，这是工程实现的技巧
        # B: batch_size, S:seq_len, D: dim
        # (B, S, D) -proj-> (B, S, D_new) -split-> (B, S, Head, W) -trans-> (B, Head, S, W)
        #           线性变换               拆成多头

        # Q: [batch_size, n_heads, len_q, d_k]
        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1, 2)
        # K: [batch_size, n_heads, len_k, d_k] # K和V的长度一定相同，维度可以不同
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1, 2)
        # V: [batch_size, n_heads, len_v(=len_k), d_v]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1, 2)

        # 因为是多头，所以mask矩阵要扩充成4维的
        # attn_mask: [batch_size, seq_len, seq_len] -> [batch_size, n_heads, seq_len, seq_len]
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)

        # context: [batch_size, n_heads, len_q, d_v], attn: [batch_size, n_heads, len_q, len_k]
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)
        # 下面将不同头的输出向量拼接在一起
        # context: [batch_size, n_heads, len_q, d_v] -> [batch_size, len_q, n_heads * d_v]
        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v)
        # 再做一个projection
        output = self.fc(context)  # [batch_size, len_q, d_model]
        return nn.LayerNorm(d_model).to(device)(output + residual), attn
    
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.GELU(),
            nn.Linear(d_ff, d_model, bias=False)
        )

    def forward(self, inputs):
        """
        inputs: [batch_size, seq_len, d_model]
        """
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).to(device)(output + residual)  # [batch_size, seq_len, d_model]
    
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        """E
        enc_inputs: [batch_size, src_len, d_model]
        enc_self_attn_mask: [batch_size, src_len, src_len]  mask矩阵(pad mask or sequence mask)
        """
        # enc_outputs: [batch_size, src_len, d_model], attn: [batch_size, n_heads, src_len, src_len]
        # 第一个enc_inputs * W_Q = Q
        # 第二个enc_inputs * W_K = K
        # 第三个enc_inputs * W_V = V
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs,
                                               enc_self_attn_mask)  # enc_inputs to same Q,K,V（未线性变换前）
        enc_outputs = self.pos_ffn(enc_outputs)
        # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn
    
class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()
        self.dec_enc_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        """
        dec_inputs: [batch_size, tgt_len, d_model]
        enc_outputs: [batch_size, src_len, d_model]
        dec_self_attn_mask: [batch_size, tgt_len, tgt_len]
        dec_enc_attn_mask: [batch_size, tgt_len, src_len]
        """
        # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len]
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)  # 这里的Q,K,V全是Decoder自己的输入
        # dec_outputs: [batch_size, tgt_len, d_model], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
        #dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs,dec_enc_attn_mask)  # Attention层的Q(来自decoder) 和 K,V(来自encoder)
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_inputs, enc_outputs, enc_outputs,dec_enc_attn_mask) 
       
        dec_outputs = self.pos_ffn(dec_outputs)  # [batch_size, tgt_len, d_model]
        return dec_outputs, dec_self_attn, dec_enc_attn  # dec_self_attn, dec_enc_attn这两个是为了可视化的
        
    

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        
        #self.src_emb = nn.Embedding(src_vocab_size, d_model)  # token Embedding
        self.conv1=nn.Conv2d(1,filters,(4,8))
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2=nn.Conv2d(filters,filters,(4,8))
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3=nn.Conv2d(filters,filters,(2,4))
        
        self.pos_emb = PositionalEncoding(d_model)  # Transformer中位置编码时固定的，不需要学习
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(encoder_n_layers)])

    def forward(self, enc_inputs):
        """
        enc_inputs: [batch_size, src_len,24,72]
        """
        #enc_outputs = self.src_emb(enc_inputs)  # [batch_size, src_len, d_model]
        
        enc_outputs = enc_inputs.reshape(-1, 24, 72).unsqueeze(1)
        #print(enc_outputs.size())
        enc_outputs = self.conv1(enc_outputs)
        #print(enc_outputs.size())
        enc_outputs = self.pool1(enc_outputs)
        enc_outputs = self.conv2(enc_outputs)
        #print(enc_outputs.size())
        enc_outputs = self.pool2(enc_outputs)
        enc_outputs = self.conv3(enc_outputs)
        #print(enc_outputs.size())
        enc_outputs = enc_outputs.reshape(-1, src_len,enc_outputs.size(-1) * enc_outputs.size(-2) * enc_outputs.size(-3))
        #[-1, src_len,30 * 18 * 6]
        #print(enc_outputs.size())
        
        #enc_outputs = enc_inputs.reshape(-1, 12,24*72)

        enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1)  # [batch_size, src_len, d_model]
        # Encoder输入序列的pad mask矩阵
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)  # [batch_size, src_len, src_len]
        enc_self_attns = []  # 画热力图等
        for layer in self.layers:  # for循环访问nn.ModuleList对象
            # 上一个block的输出enc_outputs作为当前block的输入
            # enc_outputs: [batch_size, src_len, d_model], enc_self_attn: [batch_size, n_heads, src_len, src_len]
            enc_outputs, enc_self_attn = layer(enc_outputs,
                                               enc_self_attn_mask)  # 传入的enc_outputs其实是input，传入mask矩阵是因为你要做self attention
            enc_self_attns.append(enc_self_attn)  # 这个只是为了可视化
        return enc_outputs, enc_self_attns


class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        #self.tgt_emb = nn.Embedding(tgt_vocab_size, d_model)  # Decoder输入的embed词表
        self.expand_linear = nn.Linear(tgt_vocab_size, d_model, bias=True)# 可学习query
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(decoder_n_layers)])  # Decoder的blocks

    def forward(self, dec_inputs, enc_inputs, enc_outputs):
        """
        dec_inputs: [batch_size, tgt_len]
        enc_inputs: [batch_size, src_len]
        enc_outputs: [batch_size, src_len, d_model]   # 用在Encoder-Decoder Attention层
        """
        
        dec_outputs = self.expand_linear(dec_inputs).to(device) # [batch_size, 24, d_model]
        
        #dec_outputs = dec_inputs.repeat(1,1,d_model) 
        dec_outputs = self.pos_emb(dec_outputs.transpose(0, 1)).transpose(0, 1).to(device)  
        # [batch_size, tgt_len, d_model]
        # Decoder输入序列的pad mask矩阵（这个例子中decoder是没有加pad的，实际应用中都是有pad填充的）
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).to(device)  # [batch_size, tgt_len, tgt_len]
        # Masked Self_Attention：当前时刻是看不到未来的信息的
        dec_self_attn_subsequence_mask = get_attn_subsequence_mask(dec_inputs).to(
            device)  # [batch_size, tgt_len, tgt_len]

        # Decoder中把两种mask矩阵相加（既屏蔽了pad的信息，也屏蔽了未来时刻的信息）
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequence_mask),
                                      0).to(device)  # [batch_size, tgt_len, tgt_len]; torch.gt比较两个矩阵的元素，大于则返回1，否则返回0

        # 这个mask主要用于encoder-decoder attention层
        # get_attn_pad_mask主要是enc_inputs的pad mask矩阵(因为enc是处理K,V的，求Attention时是用v1,v2,..vm去加权的，要把pad对应的v_i的相关系数设为0，这样注意力就不会关注pad向量)
        #                       dec_inputs只是提供expand的size的
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs)  # [batc_size, tgt_len, src_len]

        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
            # Decoder的Block是上一个Block的输出dec_outputs（变化）和Encoder网络的输出enc_outputs（固定）
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)
        # dec_outputs: [batch_size, tgt_len, d_model]
        return dec_outputs, dec_self_attns, dec_enc_attns

        


    
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.encoder = Encoder().to(device)
        self.decoder = Decoder().to(device)
        self.projection = nn.Linear(d_model, tgt_vocab_size, bias=False).to(device)

    def forward(self, enc_inputs, dec_inputs):
        """Transformers的输入：两个序列
        enc_inputs: [batch_size, src_len]
        修改
        enc_inputs: [batch_size, src_len, 24,72]
        dec_inputs: [batch_size, tgt_len,1]
        """
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        # 经过Encoder网络后，得到的输出还是[batch_size, src_len, d_model]
        enc_outputs, enc_self_attns = self.encoder(enc_inputs)
        # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        # dec_outputs: [batch_size, tgt_len, d_model] -> dec_logits: [batch_size, tgt_len, tgt_vocab_size]
        dec_logits = self.projection(dec_outputs)
        #return dec_logits.view(-1, dec_logits.size(-1)), enc_self_attns, dec_self_attns, dec_enc_attns
        return dec_logits, enc_self_attns, dec_self_attns, dec_enc_attns


model = Transformer().to(device)

        
class MyLoss(nn.Module):
    def __init__(self):
        super(MyLoss, self).__init__()
        
    def forward(self, pred, truth):
        """计算相关系数
        Args:
            pred ([type]): [batch_size, 24, 1]
            truth ([type]): [batch_size, 24, 1]

        Returns:
            [type]: [batch_size个相关系数的均值]
        """
        mse = 0.
        cost = torch.pow((truth.squeeze() - pred.squeeze()), 2)
        #print(cost.size())
        k = torch.arange(1, 25).unsqueeze(0).to(device)
        loss = torch.mul(cost, k)
        #print(loss.size())
        
        return  torch.mean(loss)

#criterion = MyLoss()
#criterion = nn.MSELoss()
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters())
print("if on cuda:",next(model.parameters()).is_cuda)

if on cuda: True


# Train

In [7]:
import pandas as pd 
import datetime
dfhistory = pd.DataFrame(columns = ["epoch","loss","val_loss"]) 
print("Start Training...")
nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("=========="*8 + "%s"%nowtime)
bestloss = float("inf")
epoch = 15
for epoch in range(1,epoch+1):
    #train----------------------------------------------------
  model.train()
  loss_sum = 0.0
  step = 1
  for step,(enc_inputs, dec_inputs, dec_outputs) in enumerate(loader_train, 1):
      optimizer.zero_grad()

      # 正向
      enc_inputs, dec_inputs, dec_outputs = enc_inputs.to(device), dec_inputs.to(device), dec_outputs.to(device)
      outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
      # print(outputs.size()) [32, 24, 1]
      #loss = criterion(outputs.view(-1), dec_outputs.view(-1)) 
      loss = criterion(outputs, dec_outputs) 

      # 反向
      loss.backward()
      optimizer.step()

      # print
      loss_sum += loss.item()
      if step%100 == 0:
        print(("[step = %d] loss: %.3f, ") % (step, loss_sum/step))


  #valid
  model.eval()
  val_loss_sum = 0.0
  val_step = 1

  for val_step, (enc_inputs, dec_inputs, dec_outputs) in enumerate(loader_valid, 1):
    with torch.no_grad():# 节点不进行求梯度
      enc_inputs, dec_inputs, dec_outputs = enc_inputs.to(device), dec_inputs.to(device), dec_outputs.to(device)
      outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
      #val_loss = criterion(outputs.view(-1), dec_outputs.view(-1)) 
      val_loss =criterion(outputs, dec_outputs) 

    val_loss_sum += val_loss.item()


  # log
  info = (epoch, loss_sum/step, val_loss_sum/val_step)
  dfhistory.loc[epoch-1] = info # epoch从1开始 index -1

  # print
  print(("\nEPOCH = %d, loss = %.3f,"+" val_loss = %.3f, ")% info)
  nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  print("\n"+"=========="*8 + "%s"%nowtime)
  
  if val_loss_sum <= bestloss: #换成小于等于比小于效果更好
        bestloss = val_loss_sum
        bestmodel = model
        

print('Finished Training...')



Start Training...
================================================================================2022-03-01 22:12:13

EPOCH = 1, loss = 0.014, val_loss = 0.014, 

================================================================================2022-03-01 22:12:13

EPOCH = 2, loss = 0.014, val_loss = 0.013, 

================================================================================2022-03-01 22:12:14

EPOCH = 3, loss = 0.013, val_loss = 0.013, 

================================================================================2022-03-01 22:12:14

EPOCH = 4, loss = 0.013, val_loss = 0.013, 

================================================================================2022-03-01 22:12:15

EPOCH = 5, loss = 0.013, val_loss = 0.014, 

================================================================================2022-03-01 22:12:16

EPOCH = 6, loss = 0.013, val_loss = 0.013, 

================================================================================2022-03-01 22:12:16

EPOCH

# soda微调(没啥用)

In [8]:
import pandas as pd 
import datetime
dfhistory = pd.DataFrame(columns = ["epoch","loss","val_loss"]) 
print("Start Training...")
nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("=========="*8 + "%s"%nowtime)
epoch = 2
for epoch in range(1,epoch+1):
    #train----------------------------------------------------
  model.train()
  loss_sum = 0.0
  step = 1
  for step,(enc_inputs, dec_inputs, dec_outputs) in enumerate(loader_train_sado, 1):
      optimizer.zero_grad()

      # 正向
      enc_inputs, dec_inputs, dec_outputs = enc_inputs.to(device), dec_inputs.to(device), dec_outputs.to(device)
      outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
      # print(outputs.size()) [32, 24, 1]
      #loss = criterion(outputs.view(-1), dec_outputs.view(-1)) 
      loss = criterion(outputs, dec_outputs) 

      # 反向
      loss.backward()
      optimizer.step()

      # print
      loss_sum += loss.item()
      if step%100 == 0:
        print(("[step = %d] loss: %.3f, ") % (step, loss_sum/step))


  #valid
  model.eval()
  val_loss_sum = 0.0
  val_step = 1

  for val_step, (enc_inputs, dec_inputs, dec_outputs) in enumerate(loader_valid_soda, 1):
    with torch.no_grad():# 节点不进行求梯度
      enc_inputs, dec_inputs, dec_outputs = enc_inputs.to(device), dec_inputs.to(device), dec_outputs.to(device)
      outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
      #val_loss = criterion(outputs.view(-1), dec_outputs.view(-1)) 
      val_loss =criterion(outputs, dec_outputs) 

    val_loss_sum += val_loss.item()

  # log
  info = (epoch, loss_sum/step, val_loss_sum/val_step)
  dfhistory.loc[epoch-1] = info # epoch从1开始 index -1

  # print
  print(("\nEPOCH = %d, loss = %.3f,"+" val_loss = %.3f, ")% info)
  nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  print("\n"+"=========="*8 + "%s"%nowtime)

print('Finished Training...')



Start Training...
================================================================================2022-03-01 22:12:23

EPOCH = 1, loss = 0.018, val_loss = 0.017, 

================================================================================2022-03-01 22:12:23

EPOCH = 2, loss = 0.019, val_loss = 0.017, 

================================================================================2022-03-01 22:12:23
Finished Training...


# Test

In [9]:
# ==========================================================================================
# 预测阶段
# testX, testY
def test_decoder(modle, enc_input):
  enc_input = torch.from_numpy(enc_input.astype(np.float32)).to(device)
    
  enc_outputs, enc_self_attns = bestmodel.encoder(enc_input)
  dec_input = torch.zeros(1, 0, 1).type_as(enc_input.data)# type转变
  next_symbol = 0.
  # print(enc_outputs.size())
  # print(enc_input.size())
    
  for i in range(tgt_len):
      # next_symbol两维
      # print(i)
      dec_input = torch.cat([dec_input.to(device), torch.tensor([[[next_symbol]]], dtype=enc_input.dtype).to(device)], -2)
      # print(dec_input.size())
      # print(dec_input.data)
      dec_outputs, _, _ = model.decoder(dec_input, enc_input, enc_outputs)
      # print(dec_outputs.size())
      projected = model.projection(dec_outputs)
      # print(projected.size())
      prob = projected.squeeze(0)
      # print("prob")
      # print(prob.data)
      next_word = prob.data[-1].item()
      next_symbol = next_word
      # print(next_word)
        
  dec_input = torch.cat([dec_input.to(device), torch.tensor([[[next_symbol]]], dtype=enc_input.dtype).to(device)], -2)
  dec_predict = dec_input[:, 1:]
  return dec_predict

# 每步预测的值都存入数组中
seqout = []
for i in range(len(testX)):
  testout = test_decoder(model, testX[i].reshape(1, 12, 24, 72))
  seqout.append(testout.squeeze().cpu().numpy())
  if (i+1)%100 ==0:
    print("test No." + str(i+1))

seqout = np.array(seqout)
print(seqout.shape)
cor = np.zeros((tgt_len))

for i in range(tgt_len):
  cor[i] = np.corrcoef(testY.reshape(384,tgt_len)[:,i], np.array(seqout)[:,i])[0,1]
print(cor)
print("sum :" + str(np.sum(cor)))

test No.100
test No.200
test No.300
(384, 24)
[0.96358644 0.91629727 0.85234677 0.77180521 0.68166226 0.59048313
 0.50725966 0.43788118 0.38418784 0.34469092 0.31310734 0.28147887
 0.25307009 0.22828535 0.20364979 0.17649899 0.14840572 0.12301691
 0.09899502 0.07415278 0.05103727 0.03477643 0.02939612 0.03578624]
sum :8.501857600706028


# 保存结果

In [26]:
np.save('result/seqout9',seqout)

In [ ]:
test_decoder(model, testX[0].reshape(1, 12, 24, 72))

In [ ]:
testY[0]

# mean

In [7]:
sum  = np.zeros((384, 24))
num = 10
for i in range(num):
    tmp = np.load('result/seqout'+str(i+1) + '.npy')
    #print(tmp)
    sum =sum + tmp
    
print(sum.shape)
seqout = sum/num 
print(seqout.shape)
tgt_len=24
cor = np.zeros((tgt_len))

for i in range(tgt_len):
  cor[i] = np.corrcoef(testY.reshape(384,tgt_len)[:,i], np.array(seqout)[:,i])[0,1]
print(cor)
print("sum :" + str(np.sum(cor)))

(384, 24)
(384, 24)
[0.97378202 0.9388711  0.8928915  0.83864513 0.77779451 0.71220014
 0.6494573  0.59994692 0.57515166 0.57579443 0.59315061 0.61125964
 0.61754696 0.60905918 0.58648416 0.55099123 0.50059509 0.43899815
 0.37554191 0.32122516 0.27923136 0.25239781 0.24382802 0.25073651]
sum :13.765580506731908
